In [ ]:
import sys
sys.path.append("../")

import pandas as pd
import plotly.graph_objects as go
from dateutil import parser
import datetime as dt

from api.oanda_api import OandaApi
from infrastructure.instrument_collection import instrumentCollection as ic
from db.db import DataDB

pd.set_option('display.max_rows', None)

In [ ]:
database = DataDB()

In [ ]:
ic.LoadInstrumentsDB()

In [ ]:
pairs = []
for pair, val in ic.instruments_dict.items():
    if val.ins_type == "CURRENCY" and "GBP" in pair:
        pairs.append(pair)

In [ ]:
pairs

In [ ]:
data = []
api = OandaApi()
for p in pairs:
    print(p)
    d_temp = api.get_candles_df(pair_name=p, granularity="D", count=400)
    d_temp['gain'] = ((d_temp['mid_c'] - d_temp['mid_o']) / d_temp['mid_c']) * 100
    d_temp['pair'] = p
    if '_GBP' in p:
        d_temp['gain'] = d_temp['gain'] * -1
    data.append(d_temp)
candles_df = pd.concat(data)
candles_df.reset_index(drop=True, inplace=True)
candles_df.time = candles_df.time.dt.date

In [ ]:
candles_df.tail()

In [ ]:
calendar_data = database.query_all(DataDB.CALENDAR_COLL)

In [ ]:
calendar_data

In [ ]:
calendar_data_df = pd.DataFrame.from_dict(calendar_data)

In [ ]:
calendar_data_df.head(10)

In [ ]:
calendar_data_df.date = calendar_data_df.date.dt.date

In [ ]:
calendar_data_df

In [ ]:
calendar_data_df_us = calendar_data_df[calendar_data_df.country == "united states"].copy()

In [ ]:
calendar_data_df_us 

In [ ]:
calendar_data_df_us.head(10)

In [ ]:
calendar_data_df_us.info()

In [ ]:
for col in ['actual', 'previous', 'forecast']:
    for sy in ['$' , '%', 'M', 'K']:
        calendar_data_df_us[col] = calendar_data_df_us[col].str.replace(sy, "")
    calendar_data_df_us[col].replace('', 0, inplace= True)
    calendar_data_df_us[col] = calendar_data_df_us[col].astype(float)

In [ ]:
date_match= {}
for orig in calendar_data_df_us.date.unique():
    d = orig
    tries = 0
    while d not in candles_df.time.values:
        d = d + dt.timedelta(days=1)
        tries += 1
        if tries > 5:
            print("Failed")
            break
    date_match[orig] = d

In [ ]:
date_match

In [ ]:
calendar_data_df_us['orig_date'] = calendar_data_df_us.date
calendar_data_df_us.date = [date_match[x] for x in calendar_data_df_us.date]  

In [ ]:
calendar_data_df_us['delta_prev'] = calendar_data_df_us.actual - calendar_data_df_us.previous
calendar_data_df_us['delta_fc'] = calendar_data_df_us.actual - calendar_data_df_us.forecast

In [ ]:
calendar_data_df_us.head()

In [ ]:
candles_df.head(2)

In [ ]:
candles_an = candles_df[['time', 'pair', 'gain']].copy()

In [ ]:
candles_an.head(2)

In [ ]:
merged = pd.merge(left=candles_an, right=calendar_data_df_us, left_on='time', right_on='date')

In [ ]:
merged.category.unique()

In [ ]:
c = 'unemployment rate'
df_an = merged[merged.category==c]
print(df_an[df_an.delta_prev < 0].gain.sum())
print(df_an[df_an.delta_prev >= 0].gain.sum())

In [ ]:
import plotly.express as px

In [ ]:
cat = 'unemployment rate'
df_cat = merged[merged.category==cat].copy()
for p in pairs:
    df_plot = df_cat[df_cat.pair == p]
    print(p)
    fig = px.scatter(df_plot, x="gain", y="delta_prev", trendline="ols")
    fig.show()

In [ ]:
common_dates = set(candles_an['time']).intersection(set(calendar_data_df_us['date']))
print("Common Dates:", common_dates)

In [ ]:
print(merged.head())